### Loading required libraries

In [1]:
import vanna
import os
from dotenv import load_dotenv
from vanna.remote import VannaDefault
import sqlite3

### Setting up Vanna

In [2]:
# Connecting to vanna, we are using the defaul LLM (GPT 3.5) and Vector store (pgvector) provided by vanna

load_dotenv()
vanna_api_key = os.getenv("vanna_api_key")
vanna_model_name = "vannagetsomecoffee"
vn = VannaDefault(model=vanna_model_name, api_key=vanna_api_key)

### Setting up a template sqlite db

In [3]:
# Using a template db called 'chinook' downloaded from the web
db_conn = sqlite3.connect("chinook.db")

db_cursor = db_conn.cursor()

In [8]:
# This is the query we will use to fetch ddl information to feed to vanna 
result = db_cursor.execute("SELECT type, sql FROM sqlite_master WHERE sql is not null")

# To see that the query fetches uncomment this line below
#result.fetchall()

### Adding the DB and DB's ddl information to vanna 

In [4]:
vn.connect_to_sqlite('chinook.db')

In [6]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)
Adding ddl: CREATE TABLE sqlite_sequence(name,seq)
Adding ddl: CREATE TABLE "artists"
(
    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Name] NVARCHAR(120)
)
Adding ddl: CREATE TABLE "customers"
(
    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) 
		ON DELETE NO ACTION 

In [ ]:
vn.get_training_data()

### Querying Vanna

In [7]:
# Don't use vn.ask it calls several functions : generate_sql, run_sql, and two functions for creating plots
#vn.ask(question="How many customers are from brazil?")
query = vn.generate_sql("How many customers are from brazil?")

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "customers"\r\n(\r\n    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [FirstName] NVARCHAR(40)  NOT NULL,\r\n    [LastName] NVARCHAR(20)  NOT NULL,\r\n    [Company] NVARCHAR(80),\r\n    [Address] NVARCHAR(70),\r\n    [City] NVARCHAR(40),\r\n    [State] NVARCHAR(40),\r\n    [Country] NVARCHAR(40),\r\n    [PostalCode] NVARCHAR(10),\r\n    [Phone] NVARCHAR(24),\r\n    [Fax] NVARCHAR(24),\r\n    [Email] NVARCHAR(60)  NOT NULL,\r\n    [SupportRepId] INTEGER,\r\n    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE TABLE "invoices"\r\n(\r\n    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [CustomerId] INTEGER  N

In [8]:
vn.run_sql(query)

,TotalCustomersFromBrazil
0,5


In [10]:
res = db_cursor.execute("select * from customers where country = 'Brazil';")

res.fetchall()

[(1,
  'Luís',
  'Gonçalves',
  'Embraer - Empresa Brasileira de Aeronáutica S.A.',
  'Av. Brigadeiro Faria Lima, 2170',
  'São José dos Campos',
  'SP',
  'Brazil',
  '12227-000',
  '+55 (12) 3923-5555',
  '+55 (12) 3923-5566',
  'luisg@embraer.com.br',
  3),
 (10,
  'Eduardo',
  'Martins',
  'Woodstock Discos',
  'Rua Dr. Falcão Filho, 155',
  'São Paulo',
  'SP',
  'Brazil',
  '01007-010',
  '+55 (11) 3033-5446',
  '+55 (11) 3033-4564',
  'eduardo@woodstock.com.br',
  4),
 (11,
  'Alexandre',
  'Rocha',
  'Banco do Brasil S.A.',
  'Av. Paulista, 2022',
  'São Paulo',
  'SP',
  'Brazil',
  '01310-200',
  '+55 (11) 3055-3278',
  '+55 (11) 3055-8131',
  'alero@uol.com.br',
  5),
 (12,
  'Roberto',
  'Almeida',
  'Riotur',
  'Praça Pio X, 119',
  'Rio de Janeiro',
  'RJ',
  'Brazil',
  '20040-020',
  '+55 (21) 2271-7000',
  '+55 (21) 2271-7070',
  'roberto.almeida@riotur.gov.br',
  3),
 (13,
  'Fernanda',
  'Ramos',
  None,
  'Qe 7 Bloco G',
  'Brasília',
  'DF',
  'Brazil',
  '71020-67